In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## EVA 8 code

In [2]:
import pandas as pd
!pip install datasets
from datasets import load_dataset
imdb_dataset = load_dataset('imdb')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# imbd = pd.DataFrame.from_dict(imdb_dataset['train'])

In [4]:
# imbd = imbd.drop(columns=['label'])

In [5]:
# imbd.head(5)

In [6]:
# wikitext_dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

In [7]:
# wikitext_dataset['train']

In [8]:
# import pandas as pd
# wikitext_data = pd.DataFrame.from_dict(wikitext_dataset['train'])

In [9]:
# wikitext_data

In [10]:
# snli_dataset = load_dataset('snli')

In [11]:
# snli_dataset['train']

In [12]:
#  snli_data= pd.DataFrame.from_dict(snli_dataset['train'])

In [13]:
# snli_data = snli_data.drop(columns=['premise','label'])
# snli_data = snli_data.rename(columns={'hypothesis': 'text'})

In [14]:
# snli_data

In [15]:
# cola_dataset = load_dataset('glue', 'cola')

In [16]:
# cola_dataset

In [17]:
# cola_dataset['train']['sentence']

In [18]:
# cola_data= pd.DataFrame.from_dict(cola_dataset['train'])

In [19]:
# cola_data

In [20]:
# cola_data = cola_data.drop(columns=['idx','label'])
# cola_data = cola_data.rename(columns={'sentence': 'text'})

In [21]:
# cola_data

In [22]:
# merged_df = pd.concat([imbd, wikitext_data, snli_data, cola_data], ignore_index=True)

In [23]:
# merged_df = merged_df.dropna()

In [24]:
cd /content/drive/MyDrive/sunandini

/content/drive/MyDrive/sunandini


In [25]:
# merged_df

In [26]:
# merged_df.to_csv('data.txt', sep='\t', header=False, index=False)

## BERT model:

In [27]:
# =============================================================================
# Libs
# =============================================================================
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re


# ===========================================================

In [28]:
# Transformer
# =============================================================================
def attention(q, k, v, mask = None, dropout = None):
    scores = q.matmul(k.transpose(-2, -1))
    scores /= math.sqrt(q.shape[-1])
    
    #mask
    scores = scores if mask is None else scores.masked_fill(mask == 0, -1e3)
    
    scores = F.softmax(scores, dim = -1)
    scores = dropout(scores) if dropout is not None else scores
    output = scores.matmul(v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, out_dim, dropout=0.1):
        super().__init__()
        
#        self.q_linear = nn.Linear(out_dim, out_dim)
#        self.k_linear = nn.Linear(out_dim, out_dim)
#        self.v_linear = nn.Linear(out_dim, out_dim)
        self.linear = nn.Linear(out_dim, out_dim*3)

        self.n_heads = n_heads
        self.out_dim = out_dim
        self.out_dim_per_head = out_dim // n_heads
        self.out = nn.Linear(out_dim, out_dim)
        self.dropout = nn.Dropout(dropout)
    
    def split_heads(self, t):
        return t.reshape(t.shape[0], -1, self.n_heads, self.out_dim_per_head)
    
    def forward(self, x, y=None, mask=None):
        #in decoder, y comes from encoder. In encoder, y=x
        y = x if y is None else y
        
        qkv = self.linear(x) # BS * SEQ_LEN * (3*EMBED_SIZE_L)
        q = qkv[:, :, :self.out_dim] # BS * SEQ_LEN * EMBED_SIZE_L
        k = qkv[:, :, self.out_dim:self.out_dim*2] # BS * SEQ_LEN * EMBED_SIZE_L
        v = qkv[:, :, self.out_dim*2:] # BS * SEQ_LEN * EMBED_SIZE_L
        
        #break into n_heads
        q, k, v = [self.split_heads(t) for t in (q,k,v)]  # BS * SEQ_LEN * HEAD * EMBED_SIZE_P_HEAD
        q, k, v = [t.transpose(1,2) for t in (q,k,v)]  # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        
        #n_heads => attention => merge the heads => mix information
        scores = attention(q, k, v, mask, self.dropout) # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        scores = scores.transpose(1,2).contiguous().view(scores.shape[0], -1, self.out_dim) # BS * SEQ_LEN * EMBED_SIZE_L
        out = self.out(scores)  # BS * SEQ_LEN * EMBED_SIZE
        
        return out

class FeedForward(nn.Module):
    def __init__(self, inp_dim, inner_dim, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(inp_dim, inner_dim)
        self.linear2 = nn.Linear(inner_dim, inp_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        #inp => inner => relu => dropout => inner => inp
        return self.linear2(self.dropout(F.relu(self.linear1(x)))) 

class EncoderLayer(nn.Module):
    def __init__(self, n_heads, inner_transformer_size, inner_ff_size, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(n_heads, inner_transformer_size, dropout)
        self.ff = FeedForward(inner_transformer_size, inner_ff_size, dropout)
        self.norm1 = nn.LayerNorm(inner_transformer_size)
        self.norm2 = nn.LayerNorm(inner_transformer_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        x2 = self.norm1(x)
        x = x + self.dropout1(self.mha(x2, mask=mask))
        x2 = self.norm2(x)
        x = x + self.dropout2(self.ff(x2))
        return x

class Transformer(nn.Module):
    def __init__(self, n_code, n_heads, embed_size, inner_ff_size, n_embeddings, seq_len, dropout=.1):
        super().__init__()
        
        #model input
        self.embeddings = nn.Embedding(n_embeddings, embed_size)
        self.pe = PositionalEmbedding(embed_size, seq_len)
        
        #backbone
        encoders = []
        for i in range(n_code):
            encoders += [EncoderLayer(n_heads, embed_size, inner_ff_size, dropout)]
        self.encoders = nn.ModuleList(encoders)
        
        #language model
        self.norm = nn.LayerNorm(embed_size)
        self.linear = nn.Linear(embed_size, n_embeddings, bias=False)
                
    
    def forward(self, x):
        x = self.embeddings(x)
        x = x + self.pe(x)
        for encoder in self.encoders:
            x = encoder(x)
        x = self.norm(x)
        x = self.linear(x)
        return x

# Positional Embedding
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        pe.requires_grad = False
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return self.pe[:,:x.size(1)] #x.size(1) = seq_len

In [29]:
# class SentencesDataset(Dataset):
#     #Init dataset
#     def __init__(self, sentences, vocab, seq_len):
#         dataset = self
        
#         dataset.sentences = sentences
#         dataset.vocab = vocab + ['<ignore>', '<oov>', '<mask>',"<random>"]
#         dataset.vocab = {e:i for i, e in enumerate(dataset.vocab)} 
#         dataset.rvocab = {v:k for k,v in dataset.vocab.items()}
#         dataset.seq_len = seq_len
        
#         #special tags
#         dataset.IGNORE_IDX = dataset.vocab['<ignore>'] #replacement tag for tokens to ignore
#         dataset.OUT_OF_VOCAB_IDX = dataset.vocab['<oov>'] #replacement tag for unknown words
#         dataset.MASK_IDX = dataset.vocab['<mask>'] #replacement tag for the masked word prediction task
#         dataset.RANDOM_IDX = dataset.vocab['<random>']#replacement tag for the masked word prediction task
    
    
#     #fetch data
#     def __getitem__(self, index, p_random_mask=0.15):
#         dataset = self
        
#         #while we don't have enough word to fill the sentence for a batch
#         s = []
#         while len(s) < dataset.seq_len:
#             s.extend(dataset.get_sentence_idx(index % len(dataset)))
#             index += 1
        
#         #ensure that the sequence is of length seq_len
#         s = s[:dataset.seq_len]
#         [s.append(dataset.IGNORE_IDX) for i in range(dataset.seq_len - len(s))] #PAD ok
        
#         #swap words with random words
#         s = [(random.choice(s) if random.random() < p_random_mask else w, w) for w in s]
        
#         #replace some words with RANDOM_IDX
#         s = [(dataset.RANDOM_IDX, w) if random.random() < p_random_mask else (w, dataset.IGNORE_IDX) for w in s]
        
#         return {'input': torch.Tensor([w[0] for w in s]).long(),
#                 'target': torch.Tensor([w[1] if w[0] != dataset.RANDOM_IDX else dataset.RANDOM_IDX for w in s]).long()}

#     #return length
#     def __len__(self):
#         return len(self.sentences)

#     #get words id
#     def get_sentence_idx(self, index):
#         dataset = self
#         s = dataset.sentences[index]
#         s = [dataset.vocab[w] if w in dataset.vocab else dataset.OUT_OF_VOCAB_IDX for w in s] 
#         return s


In [31]:
# =============================================================================
class SentencesDataset(Dataset):
    #Init dataset
    def __init__(self, sentences, vocab, seq_len):
        dataset = self
        
        dataset.sentences = sentences
        dataset.vocab = vocab + ['<ignore>', '<oov>', '<mask>',"<random>"]
        dataset.vocab = {e:i for i, e in enumerate(dataset.vocab)} 
        dataset.rvocab = {v:k for k,v in dataset.vocab.items()}
        dataset.seq_len = seq_len
        
        #special tags
        dataset.IGNORE_IDX = dataset.vocab['<ignore>'] #replacement tag for tokens to ignore
        dataset.OUT_OF_VOCAB_IDX = dataset.vocab['<oov>'] #replacement tag for unknown words
        dataset.MASK_IDX = dataset.vocab['<mask>'] #replacement tag for the masked word prediction task
        dataset.RANDOM_IDX = dataset.vocab['<random>']#replacement tag for the masked word prediction task
    
    
    #fetch data
    #fetch data
    def __getitem__(self, index, p_random_mask=0.15):
        dataset = self
        
        #while we don't have enough word to fill the sentence for a batch
        s = []
        while len(s) < dataset.seq_len:
            s.extend(dataset.get_sentence_idx(index % len(dataset)))
            index += 1
        
        #ensure that the sequence is of length seq_len
        s = s[:dataset.seq_len]
        [s.append(dataset.IGNORE_IDX) for i in range(dataset.seq_len - len(s))] #PAD ok
        
       #swap words with random words
        s = [(random.choice(s) if random.random() < p_random_mask else w, w) for w in s]

        
        #replace some words with RANDOM_IDX
        s = [(dataset.RANDOM_IDX, w[1]) if random.random() < p_random_mask else (w[0], dataset.IGNORE_IDX) for w in s]
        
        return {'input': torch.Tensor([w[0] for w in s]).long(),
                'target': torch.Tensor([w[1] if w[0] != dataset.RANDOM_IDX else dataset.RANDOM_IDX for w in s]).long()}


    #return length
    def __len__(self):
        return len(self.sentences)

    #get words id
    def get_sentence_idx(self, index):
        dataset = self
        s = dataset.sentences[index]
        s = [dataset.vocab[w] if w in dataset.vocab else dataset.OUT_OF_VOCAB_IDX for w in s] 
        return s

In [43]:
print("sentences      ...",sentences[1:2])

sentences      ... ['"""i am curious: yellow"" is a risible and pretentious steaming pile. it doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. as for the claim that frontal male nudity is an automatic nc-17, that isn\'t true. i\'ve seen r-rated films with male nudity. granted, they only offer some fleeting views, but where are the r-rated films with gaping vulvas and flapping labia? nowhere, because they don\'t exist. the same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. and those pretentious indie movies like the brown bunny, in which we\'re treated to the site of vincent gallo\'s throbbing johnson, but not a trace of pink visible on chloe sevigny. before crying (or implying) ""double-standard"" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresse

In [32]:
# =============================================================================
# Methods / Class
# =============================================================================
def get_batch(loader, loader_iter):
    try:
        batch = next(loader_iter)
    except StopIteration:
        loader_iter = iter(loader)
        batch = next(loader_iter)
    return batch, loader_iter

# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
pth = '/content/drive/MyDrive/data.txt'
sentences = open(pth).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]
print("sentences",sentences)
#3) create vocab if not already created
print('creating/loading vocab...')
pth = '/content/vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')

from torch.utils.data import random_split

# Split dataset into train and validation


dataset = SentencesDataset(sentences, vocab, seq_len)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders for train and validation
train_loader = torch.utils.data.DataLoader(train_dataset, **kwargs)
val_loader = torch.utils.data.DataLoader(val_dataset, **kwargs)
#data_loader = torch.utils.data.DataLoader(dataset, **kwargs)




initializing..
loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...


In [37]:
print(next(iter(dataset)))

{'input': tensor([    1,    44,  7523, 40003,  1191,  9352,    13,  1338,  1338,   271,
          282, 40003,   142,    19, 40003,     0,  2460,     1,  2714,    19]), 'target': tensor([40000, 40000, 40000, 40003, 40000, 40000, 40000, 40000, 40000, 40000,
        40000, 40003, 40000, 40000, 40003, 40000, 40000, 40000, 40000, 40000])}


In [33]:
data_loader = train_loader

In [34]:
# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = Transformer(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)



initializing model...
initializing optimizer and loss...


In [35]:
# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 1000
for it in range(n_iteration):
    
    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)
    
    #infer
    masked_input = batch['input']
    masked_target = batch['target']
    
    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)
    
    #compute the cross entropy loss 
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)
    
    #compute gradients
    loss.backward()
    
    #apply gradients
    optimizer.step()
    
    #print step
    if it % print_each == 0:
        print('it:', it, 
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))
    
    #reset gradients
    optimizer.zero_grad()
    






training...
it: 0  | loss 11.16  | Δw: 7.708
it: 10  | loss 5.64  | Δw: 2.024
it: 20  | loss 4.53  | Δw: 0.634
it: 30  | loss 4.17  | Δw: 0.302
it: 40  | loss 3.9  | Δw: 0.205
it: 50  | loss 3.64  | Δw: 0.133
it: 60  | loss 3.39  | Δw: 0.105
it: 70  | loss 3.15  | Δw: 0.094
it: 80  | loss 2.91  | Δw: 0.085
it: 90  | loss 2.68  | Δw: 0.077
it: 100  | loss 2.44  | Δw: 0.071
it: 110  | loss 2.22  | Δw: 0.066
it: 120  | loss 2.0  | Δw: 0.059
it: 130  | loss 1.79  | Δw: 0.054
it: 140  | loss 1.59  | Δw: 0.049
it: 150  | loss 1.41  | Δw: 0.045
it: 160  | loss 1.24  | Δw: 0.04
it: 170  | loss 1.09  | Δw: 0.036
it: 180  | loss 0.96  | Δw: 0.033
it: 190  | loss 0.84  | Δw: 0.03
it: 200  | loss 0.74  | Δw: 0.027
it: 210  | loss 0.65  | Δw: 0.024
it: 220  | loss 0.58  | Δw: 0.022
it: 230  | loss 0.51  | Δw: 0.02
it: 240  | loss 0.46  | Δw: 0.018
it: 250  | loss 0.42  | Δw: 0.016
it: 260  | loss 0.38  | Δw: 0.015
it: 270  | loss 0.34  | Δw: 0.014
it: 280  | loss 0.32  | Δw: 0.012
it: 290  | loss 0

In [ ]:
# Get a batch of data from the validation set
data = next(iter(val_loader))

# Use the trained model to make predictions on the batch
inputs = data['input'].cuda()
outputs = model(inputs)

# For each prediction, convert the indices back to words using the vocabulary dictionary
vocab = dataset.vocab
predictions = []

# Check length of vocabulary
print('Length of vocabulary:', len(vocab))

# Check range of values in output tensor
print('Min value in output tensor:', torch.min(outputs))
print('Max value in output tensor:', torch.max(outputs))

for i in range(len(outputs)):
    output = outputs[i].argmax(dim=1).tolist()
    prediction = [vocab.get(idx, '<unk>') for idx in output]
    predictions.append(prediction)

# Print the original sentence and the predicted sentence side by side for 10 examples
for i in range(10):
    input_sentence = ' '.join([vocab.get(idx, '<unk>') for idx in data['input'][i]])
    output_sentence = ' '.join(predictions[i])
    print('Input:    ', input_sentence)
    print('Output:    ', output_sentence)
   




In [44]:
# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )


print('end')

saving embeddings...
end


In [45]:
#mv /content/vocab.txt /content/drive/MyDrive